<a href="https://colab.research.google.com/github/rk119/F20BC/blob/main/F20BC_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Imports

In [2]:
import numpy as np
import pandas as pd

### Load the data

In [3]:
df = x_train_all = pd.read_csv("/content/drive/MyDrive/F20BC-Dataset/data_banknote_authentication.csv")

### View the first 5 instances of the data

In [4]:
df.head()

,"3.6216,8.6661,-2.8073,-0.44699,0"
0,"4.5459,8.1674,-2.4586,-1.4621,0"
1,"3.866,-2.6383,1.9242,0.10645,0"
2,"3.4566,9.5228,-4.0112,-3.5944,0"
3,"0.32924,-4.4552,4.5718,-0.9888,0"
4,"4.3684,9.6718,-3.9606,-3.1625,0"


In [5]:
df.shape

(1371, 1)

In [6]:
if len(df.columns) == 1:
    # Split the single column into multiple columns by the comma delimiter
    df = df.iloc[:, 0].str.split(',', expand=True)

# Convert the split columns to numeric types
df = df.apply(pd.to_numeric)

# Check if there are 5 columns
if df.shape[1] == 5:
    # Split into features (X) and label (Y)
    X = df.iloc[:, :-1]
    Y = df.iloc[:, -1]

# Convert X and Y to numpy arrays
X_array = X.to_numpy()
Y_array = Y.to_numpy()

In [7]:
df

,0,1,2,3,4
0,4.54590,8.16740,-2.4586,-1.46210,0
1,3.86600,-2.63830,1.9242,0.10645,0
2,3.45660,9.52280,-4.0112,-3.59440,0
3,0.32924,-4.45520,4.5718,-0.98880,0
4,4.36840,9.67180,-3.9606,-3.16250,0
...,...,...,...,...,...
1366,0.40614,1.34920,-1.4501,-0.55949,1
1367,-1.38870,-4.87730,6.4774,0.34179,1
1368,-3.75030,-13.45860,17.5932,-2.77710,1
1369,-3.56370,-8.38270,12.3930,-1.28230,1


In [10]:
len(X_array)

1371

In [11]:
len(Y_array)

1371

# Function

### Activation Functions

In [12]:
class ActivationFunctions:
  def evaluate(self,x):
    pass
  def derivate(self,x):
    pass

class Identity:
  def evaluate(self,x):
    return x
  def derivative(self,x):
    return 1

class Sigmoid(ActivationFunctions):
  def evaluate(self,x):
    return 1 / (1 + np.exp(-x))
  def derivative(self,x):
    f = self.evaluate(x)
    return f * (1-f)

class Tanh(ActivationFunctions):
  def evaluate(self,x):
    return np.tanh(x)
  def derivative(self,x):
    f = self.evaluate(x)
    return 1 - f ** 2

class relu(ActivationFunctions):
  def evaluate(self,x):
    return np.maximum(0, x)
  def derivative(self,x):
    return (x > 0).astype(float)

### Loss Functions

In [13]:
class LossFunctions:
  def evaluate(self,x):
    pass
  def derivate(self,x):
    pass

# y is predictions made by the neural network
# t is target/actual numbers corresponding to inputs
class Mse(LossFunctions):
  def evaluate(self, y, t):
    return ((t - y) ** 2).mean()
  def derivative(self, y, t):
    return 2 * (y - t) / len(y)

class BinaryCrossEntropy(LossFunctions):
  def evaluate(self, y, t):
    y_pred = np.clip(y, 1e-7, 1 - 1e-7)
    term0 = (1 - t) * np.log(1 - y_pred + 1e-7)
    term1 = t * np.log(y_pred + 1e-7)
    return - (term0 + term1).mean()

  def derivative(self, y, t):
    y_pred = np.clip(y, 1e-7, 1 - 1e-7)
    return (t / y_pred) - (1 - t) / (1 - y_pred)

class Hinge(LossFunctions):
  def evaluate(self, y, t):
    return np.maximum(0, 1 - t * y).mean()

  def derivative(self, y, t):
    return np.where(t * y < 1, -t, 0)

In [14]:
# Define test cases
test_cases = [
    {"y": np.array([1,2,3,4,5]), "t": np.array([1,2,3,5,5]), "expected": 0.2},
    {"y": np.array([1, 2, 3]), "t": np.array([2, 3, 4]), "expected": 1.0},
    # Add more test cases as needed
]

# Test function
def test_MSE_loss(loss_function, test_cases):
    for i, test_case in enumerate(test_cases, start=1):
        y, t = test_case["y"], test_case["t"]
        expected = test_case["expected"]
        result = loss_function.evaluate(y, t)
        assert np.isclose(result, expected), f"Test case {i} failed: expected {expected}, got {result}"
        print(f"Test case {i} passed.")

test_MSE_loss(Mse(), test_cases)

Test case 1 passed.
Test case 2 passed.


### Fitness Functions

In [15]:
class FitnessFunctions:
    def evaluate(self, x):
        pass

class Sphere(FitnessFunctions):
    def evaluate(self, x):
        return np.sum(x**2)

class Rastrigin(FitnessFunctions):
    def evaluate(self, x):
        A = 10 # constant that determines the amplitude of the cosine components
        return A * len(x) + np.sum(x**2 - A * np.cos(2 * np.pi * x))

class Schwefel(FitnessFunctions):
    def evaluate(self, x):
        return 418.9829 * len(x) - np.sum(x * np.sin(np.sqrt(np.abs(x))))

# Feed Forward

In [21]:
class InputLayer:
    def __init__(self, input_size):
        self.nb_nodes = input_size

    def forward(self, input_data):
        return input_data

class Layer:
    def __init__(self, input_size, nodes, activation):
        self.nb_nodes = nodes
        self.X_in = None
        # look into initialization techniques like using Xavier or He initialization
        self.W = np.random.randn(input_size, nodes)
        self.B = np.random.randn(nodes)
        self.activation = activation
        # print(input_size, 'W = ', self.W, 'B = ', self.B)

    def forward(self, input_data):
        self.X_in = input_data
        z = np.dot(input_data, self.W) + self.B
        out = self.activation.evaluate(z)
        return out

class NeuralNetwork:
    def __init__(self, configuration):
        self.layers = []
        input_size = configuration[0]
        self.layers.append(InputLayer(input_size))
        for layer_config in configuration[1:]:
            nodes, activation = layer_config
            layer = Layer(self.layers[-1].nb_nodes, nodes, activation)
            self.add(layer)

    def add(self, layer):
        self.layers.append(layer)

    def forward(self, input_data):
        for layer in self.layers:
            input_data = layer.forward(input_data)
        return input_data

def main():
    relu_activation = relu()
    sigmoid_activation = Sigmoid()

    # Sample input (1 sample, 4 features)
    # input_data = np.array([[1.0, 2.0, 3.0, 4.0],[5.0, 6.0, 3.0, 7.0],[5.0, 6.0, 3.0, 7.0]])
    input_data = X_array

    # Pick number of input nodes between 1 and num of features
    num_features = input_data.shape[1]  # number of features
    input_size = 3  # User chosen number

    # User selected features to keep # number of selected_features should be equal to input_size
    selected_features = [0, 2, 3]
    # Only keep the user selected features in the input data
    input_data = input_data[:, selected_features]

    # Configuration: [input_size, [nodes, activation], [nodes, activation], ...]
    configuration = [
        input_size,  # User chosen input size
        [4, relu_activation],  # First hidden layer
        [5, relu_activation],  # Second hidden layer
        [3, sigmoid_activation]  # Output layer
    ]

    # Initialize Neural Network with the layer configurations
    nn = NeuralNetwork(configuration)

    # Perform feedforward operation
    output = nn.forward(input_data)

    print("Output of the last layer:\n", output)

if __name__ == "__main__":
    main()

Output of the last layer:
 [[7.61467590e-01 9.99493520e-02 3.19369518e-03]
 [9.19467956e-01 5.20697644e-03 1.93222845e-06]
 [7.21047921e-01 4.07221887e-01 9.83701748e-02]
 ...
 [5.97295858e-07 1.84998868e-16 1.19204972e-22]
 [3.90409125e-05 1.17237865e-12 6.14101003e-17]
 [2.89628911e-01 1.51938875e-05 5.65019621e-06]]


# IGNORE

### Layer Initialization

In [ ]:
# class InputLayer:
#     def __init__(self, input_size):
#         self.nb_nodes = input_size

#     def forward(self, input_data):
#         return input_data

# class Layer:
#     def __init__(self, input_size, nodes, activation):
#         self.nb_nodes = nodes
#         self.X_in = None
#         self.W = np.random.randn(input_size, nodes)
#         self.B = np.random.randn(nodes)
#         self.activation = activation
#         # print(input_size, 'W = ', self.W, 'B = ', self.B)

#     def forward(self, input_data):
#         self.X_in = input_data
#         z = np.dot(input_data, self.W) + self.B
#         out = self.activation.evaluate(z)
#         return out

### Neural Network

In [ ]:
# class NeuralNetwork:
#     def __init__(self, configuration):
#         self.layers = []
#         input_size = configuration[0]
#         self.layers.append(InputLayer(input_size))
#         for layer_config in configuration[1:]:
#             nodes, activation = layer_config
#             layer = Layer(self.layers[-1].nb_nodes, nodes, activation)
#             self.add(layer)

#     def add(self, layer):
#         self.layers.append(layer)

#     def forward(self, input_data):
#         for layer in self.layers:
#             input_data = layer.forward(input_data)
#         return input_data

### Main Method

In [ ]:
# def main():
#     relu_activation = relu()
#     sigmoid_activation = Sigmoid()

#     # Sample input (1 sample, 4 features)
#     input_data = np.array([[1.0, 2.0, 3.0, 4.0],[5.0, 6.0, 3.0, 7.0],[5.0, 6.0, 3.0, 7.0]])

#     # Configuration: [input_size, [nodes, activation], [nodes, activation], ...]
#     configuration = [
#         4,  # Input size
#         [4, relu_activation],  # First hidden layer
#         [5, relu_activation],  # Second hidden layer
#         [3, sigmoid_activation]  # Output layer
#     ]


#     # Initialize Neural Network with the layer configurations
#     nn = NeuralNetwork(configuration)

#     # Perform feedforward operation
#     output = nn.forward(input_data)

#     # output is an array of feedforward being done on all samples
#     print("Output of the last layer:", output)

# if __name__ == "__main__":
#     main()

Output of the last layer: [[1.26509562e-02 1.75881331e-02 9.88595956e-01]
 [5.40614896e-06 9.99936526e-01 9.66071586e-01]
 [5.40614896e-06 9.99936526e-01 9.66071586e-01]]
